In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
import warnings 
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_excel('/kaggle/input/customer-segmentation-dataset/Online Retail.xlsx')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
def print_percent_null(data):
    null_data = data.isna().sum() / len(data)
    return null_data.apply(lambda x: f"{x:.1%}")

print_percent_null(df)

In [ ]:
df.dropna(inplace=True)

In [ ]:
print_percent_null(df)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']


new_df = df.groupby('CustomerID').agg({'InvoiceDate': lambda x: (df['InvoiceDate'].max() - x.max()).days,
                                            'InvoiceNo': 'count',
                                            'TotalPrice': 'sum'})


new_df.info()

In [ ]:
new_df.head()

In [ ]:
scaler = MinMaxScaler()
normalize_df = scaler.fit_transform(new_df)

In [ ]:
wcss = []

for each in range(1, 15):
    kmeans = KMeans(n_clusters=each)
    kmeans.fit(normalize_df)
    wcss.append(kmeans.inertia_)

plt.plot(range(1, 15), wcss)
plt.title('Elbow plot')
plt.xlabel("Number of k value (each)")
plt.ylabel("WCSS")
plt.show()

In [ ]:
kmeans1 = KMeans(n_clusters=3)
cluster_k = kmeans1.fit_predict(normalize_df)

In [ ]:
with_km_df = pd.DataFrame(normalize_df, columns=new_df.columns, index=new_df.index)
with_km_df['cluster'] = cluster_k
with_km_df.head()

In [ ]:
with_km_df['cluster'].value_counts(ascending=True)

In [ ]:
merging = linkage(normalize_df, method='ward')
dendrogram(merging, leaf_rotation=90)
plt.xlabel('Data Points')
plt.ylabel('Euclidean distance')
plt.show()

In [ ]:
hc = AgglomerativeClustering(n_clusters=3, metric='euclidean', linkage='ward')
cluster_hc = hc.fit_predict(normalize_df)

In [ ]:
with_hc_df = pd.DataFrame(normalize_df, columns=new_df.columns, index=new_df.index)
with_hc_df['cluster'] = cluster_hc
with_hc_df.head()

In [ ]:
with_hc_df['cluster'].value_counts(ascending=True)

In [ ]:
cluster_mean = with_hc_df.groupby('cluster')[['InvoiceDate', 'InvoiceNo', 'TotalPrice']].mean().reset_index()

fig, axes = plt.subplots(nrows=3, figsize=(4, 6))

sns.barplot(cluster_mean, x='cluster', y='InvoiceDate', ax=axes[0])
sns.barplot(cluster_mean, x='cluster', y='InvoiceNo', ax=axes[1])
sns.barplot(cluster_mean, x='cluster', y='TotalPrice', ax=axes[2])

plt.show()

In [ ]:
cluster_mean = with_km_df.groupby('cluster')[['InvoiceDate', 'InvoiceNo', 'TotalPrice']].mean().reset_index()

fig, axes = plt.subplots(nrows=3, figsize=(4, 6))

sns.barplot(cluster_mean, x='cluster', y='InvoiceDate', ax=axes[0])
sns.barplot(cluster_mean, x='cluster', y='InvoiceNo', ax=axes[1])
sns.barplot(cluster_mean, x='cluster', y='TotalPrice', ax=axes[2])

plt.show()